In [1]:
import pandas as pd
from betacal import BetaCalibration
from os.path import join
import glob
from IPython.display import display
from sklearn.isotonic import IsotonicRegression
from cal_methods import HistogramBinning, TemperatureScaling, evaluate, cal_results

In [2]:
def get_dataframe(dfs, column, names):
    df_res = pd.DataFrame(columns=names)

    for i in range(1, len(dfs[0]), 2):
        print("")

        name = dfs[0].iloc[i - 1]["Name"]  # Get name of method
        uncalibrated = dfs[0].iloc[i - 1][column]  # Get uncalibrated score

        row = [name, uncalibrated]  # Add scores to row

        for df in dfs:
            row.append(df.iloc[i][column])

        df_res.loc[(i - 1) // 2] = row
        # errordf.iloc[i]

    df_res.set_index('Name', inplace=True)

    return df_res


In [3]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]


In [4]:
seeds = [32, 100, 242, 376, 498]

PATH = join('..', 'logits')
files_original = [f.split("/")[-1] for f in glob.glob("../logits/*original*")]

files_blurdetail = [f.split("/")[-1] for f in glob.glob("../logits/*blurdetail*")]

files_allfilter = [f.split("/")[-1] for f in glob.glob("../logits/*allfilter*")]


In [7]:
files_dict = {"original": files_original, "blurdetail": files_blurdetail, "allfilter": files_allfilter}
model_temp_scale = dict()

for files in files_dict.keys():
    print("Generating for files: ", files)
    df_iso, model_iso = cal_results(IsotonicRegression, PATH, files_dict[files],
                         {'y_min':0, 'y_max':1}, approach = "single")
    
    df_temp_scale, model_temp_scale[files] = cal_results(TemperatureScaling, PATH, files_dict[files], approach="all")
    df_beta, model_beta = cal_results(BetaCalibration, PATH, files_dict[files], {'parameters': "abm"}, approach="single")
    df_beta_am, model_beta_am = cal_results(BetaCalibration, PATH, files_dict[files], {'parameters': "am"}, approach="single")
    df_beta_ab, model_beta_ab = cal_results(BetaCalibration, PATH, files_dict[files], {'parameters': "ab"}, approach="single")
    df_hb, model_hb = cal_results(HistogramBinning, PATH, files_dict[files], {'M': 15}, approach="single")
    
    dfs = [df_hb, df_iso, df_temp_scale, df_beta, df_beta_am, df_beta_ab]
    names = ["Name", "Uncalibrated", "Histogram Binning", "Isotonic Regression", "Temperature Scaling",
             "Beta Calibration",
             "BC am", "BC ab"]
    df_scores = ["Error", "ECE", "MCE", "Loss"]

    for s in df_scores:
        print("Results for {a}".format(a=s))
        df_get = get_dataframe(dfs, s, names)

        df_get.style.apply(highlight_min, axis=1)

        file_name = "../results/" + files + "_" + s + ".csv"
        display(df_get)
        df_get.to_csv(file_name)
    
    #for d in dfs:
    #    display(d)

Generating for files:  original
Accuracy: 65.22222222222223
Error: 34.77777777777777
ECE: 0.14978660134805577
MCE: 0.3015740371563218
Loss: 1.1995865941746626
Error 27.000000; ece 0.065515; mce 0.205816; loss 0.624973, brier 0.000000
Time taken: 0.07346606254577637 

Accuracy: 35.44444444444444
Error: 64.55555555555556
ECE: 0.418663149625063
MCE: 0.6427477521471457
Loss: 2.908929171963492
Error 47.000000; ece 0.173217; mce 0.311014; loss 1.160688, brier 0.000000
Time taken: 0.06624293327331543 

Accuracy: 27.333333333333332
Error: 72.66666666666667
ECE: 0.27914999708533283
MCE: 0.4249187862431562
Loss: 2.505881413128227
Error 51.000000; ece 0.155887; mce 0.846154; loss 1.318043, brier 0.000000
Time taken: 0.06238293647766113 



/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)


Accuracy: 33.666666666666664
Error: 66.33333333333334
ECE: 0.3376876599755552
MCE: 0.5251254218967021
Loss: 2.273300369870655
Error 43.000000; ece 0.130717; mce 0.402039; loss 1.178589, brier 0.000000
Time taken: 0.07229280471801758 

Accuracy: 41.44444444444444
Error: 58.55555555555556
ECE: 0.3224811460574468
MCE: 0.4627682868256626
Loss: 2.282380748546085
Error 42.000000; ece 0.144727; mce 0.295172; loss 1.171740, brier 0.000000
Time taken: 0.06531882286071777 

Accuracy: 45.22222222222222
Error: 54.77777777777778
ECE: 0.20219202424089117
MCE: 0.8001349866390228
Loss: 1.7643540329123384
Error 40.000000; ece 0.116721; mce 0.341982; loss 1.095450, brier 0.000000
Time taken: 0.0645902156829834 

Accuracy: 65.77777777777779
Error: 34.222222222222214
ECE: 0.13026220520337423
MCE: 0.21042844818698037
Loss: 1.1005413091937783


/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)
/run/media/bsl/Data/Work at Tartu/UT_resnet/calibration/cal_methods.py:143: RuntimeWarning: invalid value encountered in true_divide
  confs = np.max(probs, axis=1) / np.sum(probs, axis=1)


Error 27.000000; ece 0.130859; mce 0.416340; loss 0.618970, brier 0.000000
Time taken: 0.06978821754455566 

Accuracy: 51.66666666666667
Error: 48.33333333333333
ECE: 0.12634658449225955
MCE: 0.20135311089770896
Loss: 1.4556674425553058
Error 35.000000; ece 0.087915; mce 0.259095; loss 0.884166, brier 0.000000
Time taken: 0.06894040107727051 

Accuracy: 60.0
Error: 40.0
ECE: 0.12163371831178664
MCE: 0.22230591256934473
Loss: 1.1511015884850875
Error 21.000000; ece 0.131148; mce 0.628084; loss 0.663224, brier 0.000000
Time taken: 0.06558585166931152 

Accuracy: 64.77777777777777
Error: 35.22222222222223
ECE: 0.16509441817800202
MCE: 0.3290034845227101
Loss: 1.3003005017704035


...

,Uncalibrated,Histogram Binning,Isotonic Regression,Temperature Scaling,Beta Calibration,BC am,BC ab
Name,,,,,,,
blur_100,0.357704,0.135256,0.120301,0.047144,0.051982,0.055348,0.102066
blur_242,0.052750,0.078534,0.133291,0.051678,0.076514,0.076343,0.076746
blur_32,0.328226,0.159313,0.134332,0.069766,0.094095,0.085367,0.153010
blur_376,0.250281,0.191857,0.140966,0.048039,0.077081,0.092191,0.106438
blur_498,0.230988,0.155602,0.115485,0.063220,0.038516,0.053536,0.077730
detail_100,0.163005,0.129144,0.094215,0.041757,0.056476,0.058091,0.071096
detail_242,0.059517,0.124430,0.146084,0.045105,0.065397,0.066116,0.074540
detail_32,0.177154,0.191352,0.159996,0.061603,0.072280,0.066528,0.089585
detail_376,0.103230,0.205672,0.128601,0.068946,0.073378,0.073470,0.087599


Results for MCE
































,Uncalibrated,Histogram Binning,Isotonic Regression,Temperature Scaling,Beta Calibration,BC am,BC ab
Name,,,,,,,
blur_100,0.586278,0.330297,0.593290,0.233676,0.246385,0.275516,0.202532
blur_242,1.000000,0.357582,0.915531,1.000000,0.962863,0.973563,0.673073
blur_32,0.511357,0.296943,0.479085,0.538360,0.332560,0.420095,0.434828
blur_376,0.458426,0.489277,0.357795,0.252729,0.234560,0.287782,0.240308
blur_498,0.340107,0.302856,0.416947,0.139709,0.129299,0.123613,0.173949
detail_100,0.343875,0.942134,0.201770,0.083064,0.151946,0.151404,0.128152
detail_242,0.198535,0.607587,0.854363,0.384510,0.446164,0.454295,0.274355
detail_32,0.413201,0.344097,0.243315,0.138022,0.195540,0.186871,0.201810
detail_376,0.207683,0.955932,0.253293,0.184432,0.167521,0.168089,0.210607


Results for Loss
































,Uncalibrated,Histogram Binning,Isotonic Regression,Temperature Scaling,Beta Calibration,BC am,BC ab
Name,,,,,,,
blur_100,2.514598,12.647599,6.032407,1.814263,1.683297,1.685691,1.811162
blur_242,1.996772,4.786466,5.619939,1.996640,2.117229,2.101720,2.167122
blur_32,2.490333,11.981902,5.641301,1.844698,1.658273,1.650301,1.826590
blur_376,1.909256,16.082538,5.434120,1.633377,1.584340,1.532892,1.696393
blur_498,1.852212,13.863832,5.345250,1.599129,1.600413,1.587611,1.648958
detail_100,1.294445,10.741055,3.118546,1.121296,1.087855,1.096133,1.129903
detail_242,1.883584,5.351901,8.207782,1.841216,2.229766,2.221155,2.323921
detail_32,1.337565,11.152467,5.117231,1.140830,1.134416,1.087810,1.191247
detail_376,1.187317,12.955654,5.155479,1.136089,1.799981,1.759735,1.177521


In [8]:
model_temp_scale.keys()

dict_keys(['original', 'blurdetail', 'allfilter'])

In [9]:
files_original

['probs_original_sample_100_logits.p',
 'probs_original_blur_100_logits.p',
 'probs_original_blur_242_logits.p',
 'probs_original_blur_32_logits.p',
 'probs_original_blur_376_logits.p',
 'probs_original_blur_498_logits.p',
 'probs_original_detail_100_logits.p',
 'probs_original_detail_242_logits.p',
 'probs_original_detail_32_logits.p',
 'probs_original_detail_376_logits.p',
 'probs_original_detail_498_logits.p',
 'probs_original_edge_enhance_100_logits.p',
 'probs_original_edge_enhance_242_logits.p',
 'probs_original_edge_enhance_32_logits.p',
 'probs_original_edge_enhance_376_logits.p',
 'probs_original_edge_enhance_498_logits.p',
 'probs_original_sample_242_logits.p',
 'probs_original_sample_32_logits.p',
 'probs_original_sample_376_logits.p',
 'probs_original_sample_498_logits.p',
 'probs_original_sharp_100_logits.p',
 'probs_original_sharp_242_logits.p',
 'probs_original_sharp_32_logits.p',
 'probs_original_sharp_376_logits.p',
 'probs_original_sharp_498_logits.p',
 'probs_origina